# Dealing with mislabeled entries

In [1]:
import pandas as pd

In [2]:
chris_data = pd.read_csv('data/specific_cedars_programs.csv') #bc guy who gave us this, his name is chris
smaller_cedars = pd.read_csv('data/claims_selected_programs_2016_2020.csv') #cedars is fatty database og file i tried downloading was 5GB

C:\Users\audre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,27,33,37,38,40,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


merging the two because chris_data has info smaller_cedars does not

In [3]:
new = smaller_cedars.merge( chris_data, 
                     left_on = 'Program ID', right_on = 'PrgID',
                    how = 'left')

**Relevant Claims Data** --> By County: # of EE programs, # of claims, program's budget,  EE program energy savings in kwh

My goal is to make a data frame that has county as the index, and the above as columns

In [4]:
neww = new.copy()

In [5]:
relevant = neww[['County', 'Program ID', 'Budget','First Year Gross kWh',
 'First Year Gross Therm',
 'First Year Net kWh',
 'First Year Net Therm']]

In [6]:
relevant = relevant.replace(to_replace = ['Los Angeles ','San Bernardino ', 'San Joaquin ', 'Long Beach '],
            value = ['Los Angeles', 'San Bernardino', 'San Joaquin', 'Long Beach'])

In [7]:
relevant.head()

,County,Program ID,Budget,First Year Gross kWh,First Year Gross Therm,First Year Net kWh,First Year Net Therm
0,NaN,BAYREN06,352223.115500,0.000000,0.000000,0.000000,0.000000
1,NaN,BAYREN06,287466.113500,0.000000,0.000000,0.000000,0.000000
2,NaN,BAYREN06,287466.113500,0.000000,0.000000,0.000000,0.000000
3,NaN,BAYREN06,352223.115500,0.000000,0.000000,0.000000,0.000000
4,NaN,LCE-COM-001,200.320838,230.027213,-0.883901,149.517688,-0.574536


In [8]:
relevant['County'].value_counts(dropna = False)

NaN                                712876
San Mateo                           53981
Kern                                22070
Sonoma                              17113
Fresno                              12146
Santa Clara, Alameda, San Mateo     10618
Marin                                7995
Monterey                             6794
San Luis Obispo                      3797
Solano                               3697
San Francisco                        3607
Redwood                              3435
Mendocino                            2184
Los Angeles                          2030
Santa Barbara                        2010
Orange                               1372
Long Beach                           1174
Madera                                941
Napa                                  898
Riverside                             659
San Bernardino                        406
San Joaquin                           143
Ventura                               137
Yolo                              

From this, everything looks pretty normal, except maybe the Santa Clara, Alameda, San Mateo value but as long as that is consistent it's ok

FYI: the file smaller_cedars is made up of individual claims, and one energy efficiency program in a single county can have thousands of claims. So each time a County appears above, it is because there was a unique claim filed in the county. To check that: 

In [9]:
smaller_cedars['Claim ID'].value_counts()

SCE2016_Q1_0129070                   1
SCE-2017-Q2-0086997                  1
PGE-2017-Q3-3971                     1
PGE-2017-Q4-32118                    1
SCE-2018-Q4-0078008                  1
                                    ..
PGE-2017-Q4-147382                   1
PGE-2018-Q3-15536                    1
SDGE-2018-3245-10941032-11149386S    1
SDGE3223-2016-10362697-1144082       1
PGE-2020-Q2-5286                     1
Name: Claim ID, Length: 870428, dtype: int64

In [10]:
smaller_cedars.shape

(870428, 152)

whoooo all good! So, since we know that each entry is a unique claim, the following will give us our 'number of claims' column

In [42]:
cc = relevant.groupby('County', dropna = False).size() # county claims

In [41]:
relevant.shape

(870428, 7)

And to check that every claim is accounted for in cc:

In [43]:
cc.sum() == len(neww['County'])

True

:)

**BUT** weird thing happens

In [44]:
cc.head()

County
Fresno         12146
Kern           22070
Long Beach      1174
Los Angeles     2030
Madera           941
dtype: int64

There are two Los Angeles entries! However, I didn't notice this right away and just continued along.
The following is how I found the number of programs per county.

In [46]:
prg = relevant.groupby(['County', 'Program ID'], dropna = False).size().index.tolist() 
#just need to count the number of times a county shows up in this list

a,b = [list(c) for c in zip(*prg)]

ccp = pd.Series(a) # made a series so can use value_counts

cp = ccp.value_counts(dropna = False) #total programs per county

prg[0:6]

[('Fresno', 'PGE211010'),
 ('Kern', 'PGE211011'),
 ('Kern', 'SCE-13-L-002K'),
 ('Kern', 'SCG3743'),
 ('Long Beach', 'SCE-13-L-002B'),
 ('Los Angeles', 'SCE-13-L-002F')]

In [47]:
cp.head()

NaN            163
Los Angeles     11
Riverside        6
Orange           6
San Diego        6
dtype: int64

at first glance, cp doesn't have those doubles, so I still didn't notice.

In [16]:
otherstuff = relevant.groupby('County', dropna = False)

In [34]:
cbs = otherstuff.sum() #county budget and energy savings

In [35]:
cbs

,Budget,First Year Gross kWh,First Year Gross Therm,First Year Net kWh,First Year Net Therm
County,,,,,
Fresno,7.369325e+06,1.351996e+07,-4.365937e+04,1.094330e+07,-3.176930e+04
Kern,9.892057e+06,2.212044e+07,-9.845322e+04,1.883259e+07,-7.907397e+04
Long Beach,4.118075e+06,7.739129e+06,9.458570e+02,5.150099e+06,3.734130e+02
Los Angeles,1.385835e+07,1.300022e+07,2.124738e+04,9.190267e+06,1.164099e+04
Madera,1.010586e+06,1.839668e+06,-5.776970e+03,1.511291e+06,-4.371343e+03
Marin,3.284334e+06,3.036558e+06,-1.750944e+04,2.499033e+06,-1.456145e+04
Mendocino,2.175631e+06,2.998642e+06,-2.022518e+04,2.522084e+06,-1.691239e+04
Monterey,1.490090e+07,2.623487e+07,-9.608982e+04,2.149940e+07,-8.495898e+04
Napa,1.241744e+06,1.885115e+06,-8.899038e+03,1.391893e+06,-6.879284e+03


nowwwww is when I noticed those duplicates. 

In [36]:
cbs.index

Index([                         'Fresno',                            'Kern',
                            'Long Beach',                     'Los Angeles',
                                'Madera',                           'Marin',
                             'Mendocino',                        'Monterey',
                                  'Napa',                          'Orange',
                               'Redwood',                       'Riverside',
                        'San Bernardino',                       'San Diego',
                         'San Francisco',                     'San Joaquin',
                       'San Luis Obispo',                       'San Mateo',
                         'Santa Barbara', 'Santa Clara, Alameda, San Mateo',
                                'Solano',                          'Sonoma',
                                'Sutter',                         'Ventura',
                                  'Yolo',                               nan]

Now it is clear to see that some of the names of the counties have a space at the end. I tried to replace the values with a space with .replace

In [21]:
cbs.index

Index([                         'Fresno',                            'Kern',
                            'Long Beach',                     'Los Angeles',
                                'Madera',                           'Marin',
                             'Mendocino',                        'Monterey',
                                  'Napa',                          'Orange',
                               'Redwood',                       'Riverside',
                        'San Bernardino',                       'San Diego',
                         'San Francisco',                     'San Joaquin',
                       'San Luis Obispo',                       'San Mateo',
                         'Santa Barbara', 'Santa Clara, Alameda, San Mateo',
                                'Solano',                          'Sonoma',
                                'Sutter',                         'Ventura',
                                  'Yolo',                               nan]

but that did not work. I was thinking about adding up the columns somewhat manually, but figured it would be more fun to figure how to do it with pandas. ohhhhh its because it is an index not a column so .replace doesn't work

** GOTTA REPLACE IN dataframe NOT CBS BECAUSE CBS IS JUST A DATAFRAME AND DOES NOT DO ADDITION and then otherstuff is a groupby object**

In [69]:
cc = cc.rename('Total Claims')

In [68]:
cp = cp.rename('Total Programs')

In [80]:
cbs = cbs.join(cc)

In [85]:
cbs['Total Claims'] == cc

County
Fresno                             True
Kern                               True
Long Beach                         True
Los Angeles                        True
Madera                             True
Marin                              True
Mendocino                          True
Monterey                           True
Napa                               True
Orange                             True
Redwood                            True
Riverside                          True
San Bernardino                     True
San Diego                          True
San Francisco                      True
San Joaquin                        True
San Luis Obispo                    True
San Mateo                          True
Santa Barbara                      True
Santa Clara, Alameda, San Mateo    True
Solano                             True
Sonoma                             True
Sutter                             True
Ventura                            True
Yolo                             

In [87]:
cbs = cbs.join(cp)

In [95]:
cbs

,Budget,First Year Gross kWh,First Year Gross Therm,First Year Net kWh,First Year Net Therm,Total Claims,Total Programs
County,,,,,,,
Fresno,7.369325e+06,1.351996e+07,-4.365937e+04,1.094330e+07,-3.176930e+04,12146,1
Kern,9.892057e+06,2.212044e+07,-9.845322e+04,1.883259e+07,-7.907397e+04,22070,3
Long Beach,4.118075e+06,7.739129e+06,9.458570e+02,5.150099e+06,3.734130e+02,1174,1
Los Angeles,1.385835e+07,1.300022e+07,2.124738e+04,9.190267e+06,1.164099e+04,2030,11
Madera,1.010586e+06,1.839668e+06,-5.776970e+03,1.511291e+06,-4.371343e+03,941,1
Marin,3.284334e+06,3.036558e+06,-1.750944e+04,2.499033e+06,-1.456145e+04,7995,1
Mendocino,2.175631e+06,2.998642e+06,-2.022518e+04,2.522084e+06,-1.691239e+04,2184,1
Monterey,1.490090e+07,2.623487e+07,-9.608982e+04,2.149940e+07,-8.495898e+04,6794,1
Napa,1.241744e+06,1.885115e+06,-8.899038e+03,1.391893e+06,-6.879284e+03,898,1


In [102]:
cbs.to_csv('data/county_claims_programs_savings.csv')

In [103]:
test = pd.read_csv('data/county_claims_programs_savings.csv')

In [108]:
display(test.dtypes)

County                     object
Budget                    float64
First Year Gross kWh      float64
First Year Gross Therm    float64
First Year Net kWh        float64
First Year Net Therm      float64
Total Claims                int64
Total Programs              int64
dtype: object